In [1]:
import pandas as pd
import os
pd.options.display.max_colwidth = 100
os.chdir('../')
from notebooks.utils.load_data import load_psdn_ocean_transfers, load_lp_staking, load_psdn_ocean_staking

psdn_ocean_contract_address = '0x51Fa2efd62ee56a493f24AE963eAce7D0051929E'
lp_staking_contract_address = '0x08D3f2cAe4eaCDCbFb0f7FfaD07a6d00FC36Db6E' # psdnOcean-ocean lp
psdn_ocean_staking_contract_address = '0xEb1CEFa6F175B4c889582DFEea4ee155CCD5D2A5'

In [1]:
# load data psdn_ocean transfers
psdn_ocean_transfers = load_psdn_ocean_transfers()
psdn_ocean_staking = load_psdn_ocean_staking()
lp_staking = load_lp_staking()


NameError: name 'load_psdn_ocean_transfers' is not defined

In [90]:
import plotly.graph_objects as go

# Create traces
df = psdn_ocean_transfers.loc[(psdn_ocean_transfers["From"] == "0x0000000000000000000000000000000000000000") & (psdn_ocean_transfers["Method"] == "Deposit")].groupby(pd.Grouper(key='DateTime', freq='W'))['Quantity'].sum().reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['DateTime'], y=df['Quantity'],
                    mode='lines',
                    name='Ocean deposit'))

df = psdn_ocean_transfers.loc[ (psdn_ocean_transfers["Method"] == "Join Pool") & (psdn_ocean_transfers['To'] == "0xba12222222228d8ba445958a75a0704d566bf2c8")].groupby(pd.Grouper(key='DateTime', freq='W'))['Quantity'].sum().reset_index()#['To'].value_counts()
fig.add_trace(go.Scatter(x=df['DateTime'], y=df['Quantity'],
                    mode='lines',
                    name='psdnOcean join LP'))

df = psdn_ocean_transfers.loc[ (psdn_ocean_transfers["Method"] == "Stake")].groupby(pd.Grouper(key='DateTime', freq='W'))['Quantity'].sum().reset_index()
fig.add_trace(go.Scatter(x=df['DateTime'], y=df['Quantity'],
                    mode='lines',
                    name='psdnOcean single stake'))

# Edit the layout
fig.update_layout(title='psdnOcean flow',
                   xaxis_title='Day',
                   yaxis_title='Amount', 
                   autosize=True,
                    width=1000,
                    height=600,
    )
fig.show()
fig.write_image("images/ocean_deposits.png", scale=1.2)


In [55]:
# load data psdnOcean staking contract
psdn_staking_contract_data_file = '/Users/nvtrang/code/h2o/protocol-liquidity/data/export-address-token-0xEb1CEFa6F175B4c889582DFEea4ee155CCD5D2A5-psdn-stake.csv'
psdn_staking_contract = pd.read_csv(psdn_staking_contract_data_file)

# types casting
psdn_staking_contract['DateTime'] = pd.to_datetime(psdn_staking_contract['DateTime'])
psdn_staking_contract["TokenValue"] = [float(str(i).replace(",", "")) for i in psdn_staking_contract["TokenValue"]]
psdn_staking_contract.head(2)

,Txhash,Blockno,UnixTimestamp,DateTime,From,To,TokenValue,USDValueDayOfTx,ContractAddress,TokenName,TokenSymbol,Unnamed: 11
0,0x854e1b6a368cffeba39dd57de6df31d5a351f0e0a86dcb417a21d27b0372c8d9,15769069,1666024739,2022-10-17 16:38:59,0x78e869870e425a4d78bc18c46be9a6b9b8417cb5,0xeb1cefa6f175b4c889582dfeea4ee155ccd5d2a5,100.0,NaN,0x51fa2efd62ee56a493f24ae963eace7d0051929e,POSEIDON OCEAN,psdnOCEAN,NaN
1,0x0c8571483972176721d1fe8d490ba37e5936f7edbec620e864a77856769d485c,15769585,1666030955,2022-10-17 18:22:35,0xb1b1dba36512c4943f2d517d2edc256875404c5d,0xeb1cefa6f175b4c889582dfeea4ee155ccd5d2a5,39960.0,NaN,0x51fa2efd62ee56a493f24ae963eace7d0051929e,POSEIDON OCEAN,psdnOCEAN,NaN


In [91]:
# Create traces
fig = go.Figure()

df = lp_staking_contract.loc[(lp_staking_contract['TokenSymbol'] == 'OCEAN') & (lp_staking_contract['From'] ==lp_staking_contract_address.lower())].groupby(pd.Grouper(key='DateTime', freq='W'))['TokenValue'].sum().reset_index()

fig.add_trace(go.Scatter(x=df['DateTime'], y=df['TokenValue'],
                    mode='lines',
                    name='To LP stakers'))

# df = lp_staking_contract.loc[(lp_staking_contract['TokenSymbol'] == 'OCEAN') & (lp_staking_contract['From'] == '0x83304d9141c01f4dc27639f0efd0ac78efc74a38')].groupby(pd.Grouper(key='DateTime', freq='W'))['TokenValue'].sum().reset_index()

# fig.add_trace(go.Scatter(x=df['DateTime'], y=df['TokenValue'],
#                     mode='lines',
#                     name='booster'))

df = psdn_staking_contract.loc[(psdn_staking_contract['TokenSymbol'] == 'OCEAN') & (psdn_staking_contract['From'] ==psdn_ocean_staking_contract_address.lower())].groupby(pd.Grouper(key='DateTime', freq='W'))['TokenValue'].sum().reset_index()

fig.add_trace(go.Scatter(x=df['DateTime'], y=df['TokenValue'],
                    mode='lines',
                    name='To psdnOcean stakers'))

# Edit the layout
fig.update_layout(title='Ocean rewards',
                   xaxis_title='Day',
                   yaxis_title='Amount', 
                   autosize=True,
                    width=1000,
                    height=600,
    )
fig.show()
fig.write_image("images/ocean_rewards.png", scale=1.2)


In [77]:
psdn_transfers_file = '/Users/nvtrang/code/h2o/protocol-liquidity/data/export-token-0xa5b947687163fe88c3e6af5b17ae69896f4abccf-psdn-transfers.csv'
psdn_transfers = pd.read_csv(psdn_transfers_file)

psdn_transfers['DateTime'] = pd.to_datetime(psdn_transfers['DateTime'])
psdn_transfers["Quantity"] = [float(str(i).replace(",", "")) for i in psdn_transfers["Quantity"]]
psdn_transfers.head(2)
# psdn_transfers

,Txhash,Blockno,UnixTimestamp,DateTime,From,To,Quantity,Method
0,0xbdc0a909d0dffed9118471e3b45679ad1bdd63428b358a7690cc1ac0a050bc6e,14665297,1651046552,2022-04-27 08:02:32,0x7ba93e134faa36320509f2a22ef71e09cbfee74d,0xedd77d21145fa99ae03689f7341e42750f9cf33c,1000000.0,Exec Transaction
1,0x90c9011694ad5fc33edf1442369a32957676d41c100120acea811e250eb26c2e,14665854,1651053971,2022-04-27 10:06:11,0xedd77d21145fa99ae03689f7341e42750f9cf33c,0xd63107992401aac3636de4a12eb88bef4fc24b3e,2500.0,Exec Transaction


In [92]:
# Create traces
fig = go.Figure()
df = psdn_transfers.loc[psdn_transfers['From'] == '0x2337e518542db50da49f686fe3db883908c3a517'].groupby(pd.Grouper(key='DateTime', freq='W'))['Quantity'].sum().reset_index()

fig.add_trace(go.Scatter(x=df['DateTime'], y=df['Quantity'],
                    mode='lines',
                    name='To LP stakers'))

df = psdn_transfers.loc[psdn_transfers['From'] == '0x5c87da6a4947b49842c33a82bc30bac3a6811e64'].groupby(pd.Grouper(key='DateTime', freq='W'))['Quantity'].sum().reset_index()

fig.add_trace(go.Scatter(x=df['DateTime'], y=df['Quantity'],
                    mode='lines',
                    name='To psdnOcean stakers'))

# Edit the layout
fig.update_layout(title='PSDN rewards',
                   xaxis_title='Day',
                   yaxis_title='Amount', 
                   autosize=True,
                    width=1000,
                    height=600,
    )
fig.show()
fig.write_image("images/psdn_rewards.png", scale=1.2)


In [83]:
psdn_transfers.loc[psdn_transfers['Method'] == 'Get Reward']['From'].value_counts()

0x54c2e1c017941484f28386cc5e90a7dc18441bf8    165
0x484787baf344005c45e912f3117b6d7fa7507094    116
0x9b3f79495649ff97a38c11cb23a2d5d089f06585     95
0x2ae08f2cbb9a9e2d4609c487ecb1b6308718d5e8     91
0xd8031d053a2e2370c5a510c9a94aff29565a7365     86
0x2c0c00ab5442d252459287ebb238cab22f7e1780     85
0x31faeaf21c6c87ff69336455fb58ef374faa68ef     81
0x5c87da6a4947b49842c33a82bc30bac3a6811e64     45
0x0210b1ed07df594fd3aac32832a6e1690554716a     43
0x8dc8f2d579a6597c7a0e723299062387ae7d8633     39
0x2337e518542db50da49f686fe3db883908c3a517     35
0xcf0abc94eedd973b38a5be6fcd8480fc506d26b3     18
Name: From, dtype: int64

In [ ]:
# '0x5c87da6a4947b49842c33a82bc30bac3a6811e64' - psdnOcean staking
# '0x2337e518542db50da49f686fe3db883908c3a517' - lp
# '0xcf0abc94eedd973b38a5be6fcd8480fc506d26b3' -
# 0x8dc8f2d579a6597c7a0e723299062387ae7d8633-0xA5B947687163FE88C3e6af5b17Ae69896F4abccf - staking psdn

In [85]:
# ocean reward pool
# 0x90E28F3C395bE18C311f18C99646c7F2A2d288F4
